In [1]:
from threading import Thread
import os
import time

import glob
import numpy as np
import pandas as pd
import tqdm

In [8]:
BOTTLENECK_DIR = '/home/ubuntu/workspace/s3data/google_landmark/bottleneck'
TEST_DIR = '/home/ubuntu/workspace/s3data/google_landmark/bottleneck_test'
OUT_DIR = '/home/ubuntu/workspace/s3data/google_landmark/output'
#BOTTLENECK_DIR = '/home/ubuntu/workspace/kgg/google_landmark/input/sampledata/bottleneck'
#TEST_DIR = '/home/ubuntu/workspace/kgg/google_landmark/input/sampledata/bottleneck'
#OUT_DIR = '/home/ubuntu/workspace/kgg/google_landmark/input/sampledata/output'
NUM_FEATURES = 2048

In [9]:
# Read image list.
bottleneck_list = glob.glob(os.path.join(BOTTLENECK_DIR, '*.txt'))
bottleneck_base = [os.path.basename(filename) for filename in bottleneck_list]
test_list = glob.glob(os.path.join(TEST_DIR, '*.txt'))
test_base = [os.path.basename(filename) for filename in test_list]
print(bottleneck_list[:3])
print(bottleneck_base[:3])


['/home/ubuntu/workspace/kgg/google_landmark/input/sampledata/bottleneck/0000537cfe185887.txt', '/home/ubuntu/workspace/kgg/google_landmark/input/sampledata/bottleneck/00002e4e382333bf.txt', '/home/ubuntu/workspace/kgg/google_landmark/input/sampledata/bottleneck/000089733ddc8a3a.txt']
['0000537cfe185887.txt', '00002e4e382333bf.txt', '000089733ddc8a3a.txt']


In [4]:
#bottleneck = np.zeros((len(bottleneck_list), NUM_FEATURES))
#for i, filepath in enumerate(bottleneck_list):
#    with open(filepath, 'r') as f:
#        line = f.readline()
#        bottleneck[i,] = np.array([float(item) for item in line.split(',')])
#print(bottleneck.shape)

In [5]:
#tests = np.zeros((len(test_list), NUM_FEATURES))
#for i, filepath in enumerate(test_list):
#    with open(filepath, 'r') as f:
#        line = f.readline()
#        tests[i,] = np.array([float(item) for item in line.split(',')])
#print(tests.shape)

In [10]:
# Compute distance
DATASIZE_WORKER = 1
THREADS_MAX = 500
lt = np.hstack([np.arange(0, len(test_list), DATASIZE_WORKER), len(test_list)])
#dist_all = pd.DataFrame(np.zeros([len(test_list),  len(bottleneck_list)]), index=test_base, columns=bottleneck_base)

# process per data chunk.
def corr_worker(idx_from, idx_to):
    dist_worker = pd.DataFrame(np.zeros([idx_to - idx_from, len(bottleneck_list)]),
                               index=test_base[idx_from:idx_to],
                               columns=bottleneck_base)
    for i in range(idx_from, idx_to):
        with open(test_list[i], 'r') as f:
            line = f.readline()
            test_i = np.array([float(item) for item in line.split(',')])
        for j, filepath in enumerate(bottleneck_list):
            with open(filepath, 'r') as f:
                line = f.readline()
                bottleneck_j = np.array([float(item) for item in line.split(',')])
            #dist_all.iloc[i, j] = np.linalg.norm(test_i - bottleneck_j)
            dist_worker.iloc[i - idx_from, j] = np.linalg.norm(test_i - bottleneck_j)
    print('test_{:09d}_{:09d}.csv'.format(idx_from, idx_to))
    # dist_worker.to_csv(os.path.join(OUT_DIR, 'test_{:09d}_{:09d}.csv'.format(idx_from, idx_to)))
    dist_worker.to_csv(os.path.join(OUT_DIR, 'test_{:09d}.csv'.format(idx_from)))

t1 = time.time()
threads = []
for i in tqdm.tqdm(range(len(lt) - 1), total=len(lt) - 1):
    thread = Thread(target=corr_worker, args=(lt[i], lt[i + 1]))
    thread.start()
    threads.append(thread)
    if len(threads) > THREADS_MAX:
        for thread in threads:
            thread.join()
        threads = []
for thread in threads:
    thread.join()

t2 = time.time()
 
# print(dist_all)


 40%|████      | 4/10 [00:02<00:03,  1.62it/s]

test_000000000_000000001.csv
test_000000001_000000002.csv



 60%|██████    | 6/10 [00:03<00:02,  1.66it/s]

test_000000003_000000004.csv



 70%|███████   | 7/10 [00:04<00:01,  1.73it/s]

test_000000002_000000003.csv
test_000000004_000000005.csv



 90%|█████████ | 9/10 [00:05<00:00,  1.73it/s]

test_000000006_000000007.csv
test_000000005_000000006.csv


100%|██████████| 10/10 [00:05<00:00,  1.80it/s]


test_000000007_000000008.csv
test_000000008_000000009.csv
test_000000009_000000010.csv
